# Marine EOV Broker

This a test notebook to explore the interactions between the EOV Broker and the VLIZ ERDDAP server.

Still TODO:
  - Need to fix the broker query limit to the ERDDAP server. Too many: "Erddap server answered with HTTP code 429 and reason Too Many Requests"

In [1]:
## Uncomment the below line to force reinstall the broker lib from the forked GIT repo's latest release:

# ! pip install --upgrade --force-reinstall git+https://github.com/vliz-be-opsci/marine-eov-broker

In [2]:
from marine_eov_broker import MarineRiBroker
import logging
import matplotlib.pyplot as plt

logger = logging.getLogger()
logger.setLevel(logging.INFO) 

## Start the broker

It will take some time (though it still needs improvements on performances). This is because the broker will :
* load vocabularies upon startup
* load erddap datasets metadata from all erddap servers

Currently this only fetches 3 datasets from the VLIZ erddap server in order to reduce the time spent searching for data. IF you want to search ALL datasets on the VLIZ server replace the list with a None. 

In [3]:
%%time
try:
    MarineRiBroker.EOV_LIST = ['EV_OXY', 'EV_SEATEMP', 'EV_SALIN']
    broker = MarineRiBroker.MarineBroker(erddap_servers={"https://erddap.eurobis.org//erddap":['lyumkis','benthos_azov_sea','benthos_blackseacruises_2003_2011']})
except Exception as error:
    print(error)
    x = error

INFO:root:Querying vocabulary server for EOV : EV_OXY
INFO:root:Querying vocabulary server for EOV : EV_SEATEMP
INFO:root:Querying vocabulary server for EOV : EV_SALIN


CPU times: user 57.6 ms, sys: 4.76 ms, total: 62.3 ms
Wall time: 6.1 s


In [26]:
broker.datasets_list

[('https://erddap.eurobis.org//erddap', 'lyumkis'),
 ('https://erddap.eurobis.org//erddap', 'benthos_azov_sea'),
 ('https://erddap.eurobis.org//erddap', 'benthos_blackseacruises_2003_2011')]

In [44]:
broker.vocabularies.keys() 

dict_keys(['EV_OXY', 'EV_SALIN', 'EV_SEATEMP'])

## Create a request to the broker :
The user must provide the EOVs, min/max date/lat/lon, output format desired.

When creating a query, the broker :
* first looks at every dataset to see if they match any eov requested by the user
* then checks if the datasets match the time/bbox requested by the user

In [4]:
start_date = "1900-01-01"
end_date = "2022-01-02"
# North-east Atlantic Ocean
min_lon = 0
min_lat = 0
max_lon = 90
max_lat = 90

logger.setLevel(logging.DEBUG)

In [5]:
%%time
response = broker.submit_request(["EV_SALIN", "EV_OXY", "EV_SEATEMP"], 
                                 start_date,
                                 end_date,
                                 min_lon,
                                 min_lat,
                                 max_lon,
                                 max_lat,
                                 "nc"
                                 )

DEBUG:marine_eov_broker.MarineRiBroker:Looking for eovs in benthos_azov_sea took 0.0002636909484863281 seconds with result : ['Temp']
DEBUG:marine_eov_broker.MarineRiBroker:Looking for eovs in lyumkis took 0.00011277198791503906 seconds with result : ['Temp']
DEBUG:marine_eov_broker.ErddapMarineRI:Will check spatiotemporal constraints from query https://erddap.eurobis.org//erddap/tabledap/benthos_azov_sea.csv?time&time%3E=1900-01-01&time%3C=2022-01-02&latitude%3E=0.0&latitude%3C=90.0&longitude%3E=0.0&longitude%3C=90.0&orderByLimit(%22time/6months,1%22)
DEBUG:marine_eov_broker.MarineRiBroker:Looking for eovs in benthos_blackseacruises_2003_2011 took 0.00011467933654785156 seconds with result : ['Salinity', 'WC_Temp']
DEBUG:marine_eov_broker.ErddapMarineRI:Will check spatiotemporal constraints from query https://erddap.eurobis.org//erddap/tabledap/lyumkis.csv?time&time%3E=1900-01-01&time%3C=2022-01-02&latitude%3E=0.0&latitude%3C=90.0&longitude%3E=0.0&longitude%3C=90.0&orderByLimit(%22tim

CPU times: user 117 ms, sys: 445 µs, total: 117 ms
Wall time: 299 ms


### Broker datasets
Let's take a close look at the dataset class:

In [53]:
broker.datasets[1].data_url

'https://erddap.eurobis.org//erddap/tabledap/lyumkis'

In [59]:
broker.datasets[1].metadata_url

'https://erddap.eurobis.org//erddap/info/lyumkis/index.csv'

In [60]:
broker.datasets[1].metadata 

,Row Type,Variable Name,Attribute Name,Data Type,Value
0,attribute,NC_GLOBAL,AccConstrDescription,String,"This license lets others distribute, remix, tw..."
1,attribute,NC_GLOBAL,AccConstrDisplay,String,This dataset is licensed under a Creative Comm...
2,attribute,NC_GLOBAL,AccConstrEN,String,Attribution (CC BY)
3,attribute,NC_GLOBAL,AccessConstraint,String,Attribution (CC BY)
4,attribute,NC_GLOBAL,AccessConstraints,String,None
...,...,...,...,...,...
171,attribute,WaterAbund,long_name,String,SDN:P01::SDBIOL01
172,attribute,WaterAbund,sdn_parameter_urn,String,SDN:P01::SDBIOL01
173,attribute,WaterAbund,units,String,None
174,attribute,WaterAbund,units_uri,String,None


In [6]:
response.queries

,query_url,AccConstrDescription,AccConstrDisplay,AccConstrEN,AccessConstraint,AccessConstraints,Acronym,BrackishFlag,CDate,cdm_data_type,...,VersionDay,VersionMonth,VersionName,VersionYear,VlizCoreFlag,Westernmost_Easting,query_object,EV_OXY,EV_SEATEMP,EV_SALIN
benthos_azov_sea,https://erddap.eurobis.org//erddap/tabledap/be...,"This license lets others distribute, remix, tw...",This dataset is licensed under a Creative Comm...,Attribution (CC BY),Attribution (CC BY),None,None,0,2013-09-30,Other,...,None,None,None,None,1,34.84,<marine_eov_broker.MarineRiBroker.ErddapReques...,,Temp,
lyumkis,https://erddap.eurobis.org//erddap/tabledap/ly...,"This license lets others distribute, remix, tw...",This dataset is licensed under a Creative Comm...,Attribution (CC BY),Attribution (CC BY),None,None,0,2010-11-03,Other,...,3,11,1,2010,1,29.7,<marine_eov_broker.MarineRiBroker.ErddapReques...,,Temp,
benthos_blackseacruises_2003_2011,https://erddap.eurobis.org//erddap/tabledap/be...,"This license lets others distribute, remix, tw...",This dataset is licensed under a Creative Comm...,Attribution (CC BY),Attribution (CC BY),None,None,0,2016-04-25,Other,...,None,None,None,None,1,27.53,<marine_eov_broker.MarineRiBroker.ErddapReques...,,WC_Temp,Salinity


In [7]:
# %%time
# response = broker.submit_request(["EV_SALIN", "EV_OXY", "EV_SEATEMP", "EV_CO2", "EV_CHLA"], 
#                                  start_date,
#                                  end_date,
#                                  min_lon,
#                                  min_lat,
#                                  max_lon,
#                                  max_lat,
#                                  "nc"
#                                  )

## Query Response

In [8]:
xx = response.get_dataset('lyumkis')

In [9]:
xx.metadata

,Row Type,Variable Name,Attribute Name,Data Type,Value
0,attribute,NC_GLOBAL,AccConstrDescription,String,"This license lets others distribute, remix, tw..."
1,attribute,NC_GLOBAL,AccConstrDisplay,String,This dataset is licensed under a Creative Comm...
2,attribute,NC_GLOBAL,AccConstrEN,String,Attribution (CC BY)
3,attribute,NC_GLOBAL,AccessConstraint,String,Attribution (CC BY)
4,attribute,NC_GLOBAL,AccessConstraints,String,None
...,...,...,...,...,...
171,attribute,WaterAbund,long_name,String,SDN:P01::SDBIOL01
172,attribute,WaterAbund,sdn_parameter_urn,String,SDN:P01::SDBIOL01
173,attribute,WaterAbund,units,String,None
174,attribute,WaterAbund,units_uri,String,None


In [10]:
xx.found_eovs

{'EV_SEATEMP': ['Temp']}

In [11]:
xx.data_url

'https://erddap.eurobis.org//erddap/tabledap/lyumkis'

You can do a simple search on the ERDDAP metadata to find whatever text you're looking for. 

In [12]:
xx.metadata[xx.metadata['Value'].str.startswith('SDN:P01', na=False)]

,Row Type,Variable Name,Attribute Name,Data Type,Value
164,attribute,Temp,long_name,String,SDN:P01::TEMPPR01
165,attribute,Temp,sdn_parameter_urn,String,SDN:P01::TEMPPR01
171,attribute,WaterAbund,long_name,String,SDN:P01::SDBIOL01
172,attribute,WaterAbund,sdn_parameter_urn,String,SDN:P01::SDBIOL01


## Results

The interesting part !
The broker provides a BrokerResponse object. It contains the variable **queries** which is a Pandas DataFrame.

The pandas DataFrame contains all the global attributes, query URL and ErddapRequest object for each dataset found for the user request.

In [13]:
response.queries

,query_url,AccConstrDescription,AccConstrDisplay,AccConstrEN,AccessConstraint,AccessConstraints,Acronym,BrackishFlag,CDate,cdm_data_type,...,VersionDay,VersionMonth,VersionName,VersionYear,VlizCoreFlag,Westernmost_Easting,query_object,EV_OXY,EV_SEATEMP,EV_SALIN
benthos_azov_sea,https://erddap.eurobis.org//erddap/tabledap/be...,"This license lets others distribute, remix, tw...",This dataset is licensed under a Creative Comm...,Attribution (CC BY),Attribution (CC BY),None,None,0,2013-09-30,Other,...,None,None,None,None,1,34.84,<marine_eov_broker.MarineRiBroker.ErddapReques...,,Temp,
lyumkis,https://erddap.eurobis.org//erddap/tabledap/ly...,"This license lets others distribute, remix, tw...",This dataset is licensed under a Creative Comm...,Attribution (CC BY),Attribution (CC BY),None,None,0,2010-11-03,Other,...,3,11,1,2010,1,29.7,<marine_eov_broker.MarineRiBroker.ErddapReques...,,Temp,
benthos_blackseacruises_2003_2011,https://erddap.eurobis.org//erddap/tabledap/be...,"This license lets others distribute, remix, tw...",This dataset is licensed under a Creative Comm...,Attribution (CC BY),Attribution (CC BY),None,None,0,2016-04-25,Other,...,None,None,None,None,1,27.53,<marine_eov_broker.MarineRiBroker.ErddapReques...,,WC_Temp,Salinity


**Or just the list of datasets ID**

In [14]:
response.get_datasets_list()

['benthos_azov_sea', 'lyumkis', 'benthos_blackseacruises_2003_2011']

### Access a dataset with its dataset ID

In [15]:
dataset_id = response.get_datasets_list()[0]
print(dataset_id)

benthos_azov_sea


### Get the description of the EOVs found variables in the dataset

In [16]:
dataset_id = 'lyumkis'
response.get_dataset_EOVs_list(dataset_id)

{'EV_SALIN': '', 'EV_OXY': '', 'EV_SEATEMP': 'Temp'}

### Get the query URL for the dataset ID

In [17]:
response.get_dataset_query_url(dataset_id)

'https://erddap.eurobis.org//erddap/tabledap/lyumkis.nc?time%2Clatitude%2Clongitude%2CTemp&time%3E=1900-01-01&time%3C=2022-01-02&latitude%3E=0.0&latitude%3C=90.0&longitude%3E=0.0&longitude%3C=90.0'

In [18]:
response.__class__

marine_eov_broker.MarineRiBroker.BrokerResponse

### Execute a query & get the result as a Pandas DataFrame...

In [19]:
df = response.dataset_to_pandas_dataframe(dataset_id)
df

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): erddap.eurobis.org:443
DEBUG:urllib3.connectionpool:https://erddap.eurobis.org:443 "GET //erddap/tabledap/lyumkis.nc?time%2Clatitude%2Clongitude%2CTemp&time%3E=1900-01-01&time%3C=2022-01-02&latitude%3E=0.0&latitude%3C=90.0&longitude%3E=0.0&longitude%3C=90.0 HTTP/1.1" 200 None


,time,latitude,longitude,Temp
row,,,,
0,2005-01-07 05:35:00,45.720001,31.690001,0
1,2005-01-07 05:35:00,45.720001,31.690001,0
2,2005-01-07 05:35:00,45.720001,31.690001,0
3,2005-01-07 05:35:00,45.720001,31.690001,0
4,2005-01-07 05:35:00,45.720001,31.690001,0
...,...,...,...,...
26352,1992-09-12 02:28:00,42.880001,40.400002,22
26353,1992-09-12 02:28:00,42.880001,40.400002,22
26354,1992-09-12 02:28:00,42.880001,40.400002,22


### ... or an Xarray dataset

In [20]:
ds = response.dataset_to_xarray(dataset_id)
ds

<xarray.Dataset>
Dimensions:    (row: 26357)
Dimensions without coordinates: row
Data variables:
    time       (row) datetime64[ns] ...
    latitude   (row) float32 ...
    longitude  (row) float32 ...
    Temp       (row) object ...
Attributes: (12/76)
    AccConstrDescription:      This license lets others distribute, remix, tw...
    AccConstrDisplay:          This dataset is licensed under a Creative Comm...
    AccConstrEN:               Attribution (CC BY)
    AccessConstraint:          Attribution (CC BY)
    AccessConstraints:         None
    Acronym:                   None
    ...                        ...
    VersionDay:                3
    VersionMonth:              11
    VersionName:               1
    VersionYear:               2010
    VlizCoreFlag:              1
    Westernmost_Easting:       29.7

### Only retrieve a specific EOV :

In [21]:
# ds = response.dataset_to_pandas_dataframe(dataset_id, "EV_OXY")
# # ds

### Download a dataset as a NetCDF file

In [22]:
response.dataset_to_file_download(dataset_id, "nc")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): erddap.eurobis.org:443
DEBUG:urllib3.connectionpool:https://erddap.eurobis.org:443 "GET //erddap/tabledap/lyumkis.nc?time%2Clatitude%2Clongitude%2CTemp&time%3E=1900-01-01&time%3C=2022-01-02&latitude%3E=0.0&latitude%3C=90.0&longitude%3E=0.0&longitude%3C=90.0 HTTP/1.1" 200 None


True